# Using the basicrobot 
See the project it.unibo.basicrobot2020

## Interaction with the Virtual Robot
The virtual robot is described in 
<a href="http://htmlpreview.github.com/?https://github.com/anatali/iss2020Lab/blob/master/it.unibo.robot2020/userDocs/LabVirtualRobot.html" target="web">
LabVirtualRobot</a>

In [ ]:
##############################################################
# clientToWenv.py
# sendToVirtualRobot : sends a command in output
# read               : acquires data from input
##############################################################
import socket
import time
import json

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
virtualRobotPort = 8999
sep = ';'   #required by the Soffritti virtual robot 'language'

goForwardMsg  = '{"type": "moveForward",   "arg": -1  }'
goBackwardMsg = '{"type": "moveBackward",  "arg": -1  }'
turnLeftMsg   = '{"type": "turnLeft",      "arg": 400 }'
turnRightMsg  = '{"type": "turnRight",     "arg": 400 }'
haltMsg       = '{"type": "alarm",         "arg": 0   }'

def connect(port) :
    server_address = ('localhost', port)
    sock.connect(server_address)    
    print("CONNECTED WITH the virtual robot" , server_address)

def sendToVirtualRobot( message ) :
    print("forward ", message)
    msgJson = json.loads( message )
    msg     = sep+ json.dumps(msgJson)+ sep 
    byt=msg.encode()    #required in Python3
    sock.send( byt )

def work() :
    sendToVirtualRobot( goForwardMsg ) 
    time.sleep(1)
    sendToVirtualRobot( haltMsg ) 

def read() :
    BUFFER_SIZE = 1024
    data = sock.recv(BUFFER_SIZE)
    print( "received data:", data )

def terminate() :
    sock.close()
    print("BYE")

###########################################    
connect(virtualRobotPort)
work()
read()
terminate()

## Interaction with the basicrobot qactor
See the model: <a href="../src/basicrobot.qak" target="code">basicrobot.qak</a>.
<ol>
    <li>
        Activate the virtual robot and oper a browser on <tt>localhost:8090</tt>
    </li>
    <li>Launch the <tt>basicrobot</tt> as a 'standalone' application (microservice)
<pre>
cd C:\..\it.unibo.basicrobot2020-1.0\bin
it.unibo.basicrobot2020.bat
</pre>
    </li>
    <li>
        Execute the python code hereunder to move the robot
    </li>
    <li>
        Look at the received data when the (virtual) robot hits an obstacle
    </li>
</ol>


In [2]:
##############################################################
# sendDispatch       : sends a command in output
# read               : acquires data from input
##############################################################
import socket
import time



basicRobotPort = 8018  
smartRobotPort = 8020  

robotName      = "basicrobot"

goForwardMsg  = "msg(cmd,dispatch,python,"+ robotName +",cmd(w),1)"
goBackwardMsg = "msg(cmd,dispatch,python,"+ robotName +",cmd(s),1)" 
turnLeftMsg   = "msg(cmd,dispatch,python,"+ robotName +",cmd(a),1)"  
turnRightMsg  = "msg(cmd,dispatch,python,"+ robotName +",cmd(d),1)"  
haltMsg       = "msg(cmd,dispatch,python,"+ robotName +",cmd(h),1)"

sock          = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

def connect(port) :
    server_address = ('localhost', port)
    sock.connect(server_address)    
    print("CONNECTED WITH , server_address")

def sendDispatch( message ) :
    print("forward ", message)
    msg = message + "\n"
    byt=msg.encode()    #required in Python3
    sock.send(byt)

def terminate() :
    sock.close()    #qak infrastr receives a msg null
    print("BYE")

def read() :
    sock.settimeout(1)
    try:
        rec = sock.recv(100) # try to receive 100 bytes
        print("RECEIVED:" , rec )
    except socket.timeout: # fail after 1 second of no activity
        print("Didn't receive data! ")
    finally:
        terminate()

def console() :  
    cmd = str(input())
    while( cmd != "z" ) :
        if( cmd == "w" ) :
            sendDispatch( goForwardMsg ) 
        if( cmd == "h" ) :
            sendDispatch( haltMsg ) 
        cmd = str(input())
          
##################################################
connect(basicRobotPort)
console()
read() 

CONNECTED WITH , server_address
w
forward  msg(cmd,dispatch,python,basicrobot,cmd(w),1)
z
RECEIVED: b'msg(obstacle,event,tcp,none,obstacle(pantry),8)\n'
BYE
